<a href="https://colab.research.google.com/github/renatopuga/rnaseq-chr22/blob/main/rnaseq_chr22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNAseq Google Colab
> Prof. Renato Puga - renatopuga@gmail.com

Aula prática de `RNAseq` da Pós-graduação de Bioinformática aplicada a Genômica Médica do Hospital Albert Einstein.

# Objetivo 

Executar um pipeline de RNAseq para seis amostras simuladas*, preparando a referência do genoma, alinhando os arquivos fastqs até a tabela de contagem de reads.

Cada uma das amostras contém 30 mil reads do chr22 do genoma humano hg19.

>\* As sequências foram geradas previamente com o comando `rsem-simulate-reads` do pacote RSEM.


# STAR (Spliced Transcripts Alignment to a Reference)
> https://academic.oup.com/bioinformatics/article/29/1/15/272537

STAR é um alinhador projetado para abordar especificamente muitos dos desafios do mapeamento de dados RNA-seq usando uma estratégia para contabilizar alinhamentos emendados.



## Clonar Repositório git

Ao criar um repositório no GitHub, ele passa a existir como um repositório remoto. É possível clonar o repositório para criar uma cópia local no Google Drive e sincronizar entre os dois locais.

Removendo arquivos e diretórios anteriores.

In [ ]:
%%bash
cd /content/
rm -rf rnaseq-chr22
pwd

In [ ]:
! git clone https://github.com/renatopuga/rnaseq-chr22

## Entrando no Diretório Clonado

Após clonar o repositório git, temos que entrar no diretório e seguir alguns passos de instalação para executar o pipeline STAR.

In [ ]:
%cd /content/rnaseq-chr22/

## Download e Instalação STAR e RSEM

- mkdir -p # ignora se o diretório existe
- wget -c # volta o download a partir de onde parou.

- TAR (agrupar) GZ (compactar) # igual ao ZIP no Windows
- tar -zxvf # z: zipado; x: extrair; v: verbose f: files

## STAR download

In [ ]:
%%bash
mkdir -p apps
cd apps
wget -c https://github.com/alexdobin/STAR/archive/2.7.8a.tar.gz
tar -zxvf 2.7.8a.tar.gz

## RSEM download

In [ ]:
%%bash
cd /content/rnaseq-chr22/apps/
wget -c https://github.com/deweylab/RSEM/archive/v1.3.3.tar.gz
tar -zxvf v1.3.3.tar.gz

In [ ]:
%%bash
cd /content/rnaseq-chr22/apps/RSEM-1.3.3
make
cd ..


# Gerando Index dos arquivos FASTA

In [ ]:
%cd /content/rnaseq-chr22/reference/

# Instalando bowtie2
> http://bowtie-bio.sourceforge.net/bowtie2/index.shtml

Bowtie 2 is an ultrafast and memory-efficient tool for aligning sequencing reads to long reference sequences.

In [ ]:
!wget -c https://sourceforge.net/projects/bowtie-bio/files/bowtie2/2.4.2/bowtie2-2.4.2-sra-linux-x86_64.zip/download

# Descompactando (unzip)

Descompactando arquivo .ZIP (download)

In [ ]:
!unzip download

# Variável de Ambiente
Vamos adicionar o caminho do bowtie2 no PATH.

In [ ]:
%cd /content/rnaseq-chr22/reference/bowtie2-2.4.2-sra-linux-x86_64/
%ls

In [ ]:
# colocando o caminho do diretório bowtie2 na variável PATH
!export PATH=$PWD/bowtie2-2.4.2-sra-linux-x86_64:$PATH

## Executando script makeRef.sh para indexar o arquivo chr22.fa

In [ ]:
%%bash
cd /content/rnaseq-chr22/reference/
sh makeRef.sh

In [ ]:
# o arquivo rsem.grp deve estar na lista
%ls rsem/

# Voltando para o diretório principal

In [ ]:
%cd /content/rnaseq-chr22/

# Executar STAR

In [ ]:
%%bash
cd /content/rnaseq-chr22/
sh run.star.rsem.sh

# Entrar no diretório de Resultados RNASEQ_data

In [ ]:
# listando diretórios de output STAR e RSEM
%cd /content/rnaseq-chr22/RNASEQ_data
%ls 

In [ ]:
# listando arquivos do diretorio A01
%cd /content/rnaseq-chr22/RNASEQ_data
%ls -lh A01

In [ ]:
# listando arquivos do diretorio rsem.A01
%cd /content/rnaseq-chr22/RNASEQ_data
%ls -lh rsem.A01

In [ ]:
# copiando resultado de contagem de cada amostra e salvando com o 
# nome da amostra correspondente de cada arquivo.
%%bash
cd /content/rnaseq-chr22/RNASEQ_data 
mkdir -p gene-level
ls -d1 rsem.*  | awk '{ print("cp -v",$1"/rsem.genes.results gene-level/"$1)}'  | sh

In [ ]:
# utilizando script R para juntar amostras por gene
%%bash
cd /content/rnaseq-chr22/RNASEQ_data
time R --slave --file=../run.merge.files.R --args gene-level 5 gene-level-5

In [ ]:
# listando tabela merge STAR com todas as amostras por gene 
# Apenas genes do chr22
%%bash
cd /content/rnaseq-chr22/RNASEQ_data
head merge-table-STAR-gene-level-5-50x.txt